In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install wandb

In [3]:
!pip install git+https://github.com/cosmoquester/transformers-tf-finetune.git

  Cloning https://github.com/cosmoquester/transformers-tf-finetune.git to /tmp/pip-req-build-i64cbz0g
  Running command git clone -q https://github.com/cosmoquester/transformers-tf-finetune.git /tmp/pip-req-build-i64cbz0g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.2 MB/s 
     |████████████████████████████████| 462 kB 63.9 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 6.5 MB 43.7 MB/s 
     |████████████████████████████████| 596 kB 52.1 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
  Created wheel for transformers-tf-finetune: filename=transformers_tf_finetune-0.0.1-py3-none-any.whl size=9872 sha256=52cfe8af6a33f7feb243db23015e81adc1e0c5d61430f917c9172bda7713f2fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-ip37w4vw/wheel

In [4]:
import csv
import random
import urllib.request

import tensorflow as tf
from transformers import AdamWeightDecay, AutoTokenizer

from transformers_tf_finetune.models import TFBartForSequenceMultiClassification
from transformers_tf_finetune.utils import LRScheduler, get_device_strategy, path_join, set_random_seed

# import wandb
# from wandb.keras import WandbCallback

import pandas as pd

# Config

In [5]:
#: transformers pretrained path
pretrained_model = "cosmoquester/bart-ko-small"
# pretrained_model = "hyunwoongko/kobart"
#: pretrained tokenizer fast pretrained path
pretrained_tokenizer = "cosmoquester/bart-ko-small"
# pretrained_tokenizer = "hyunwoongko/kobart"
#: load from pytorch weight
from_pytorch = True
#: use huggingface credential for private model
use_auth_token = ""

train_dataset_path = "/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/data/train.csv"
dev_dataset_path = "/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/data/test.csv"
test_dataset_path = "/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/data/test.csv"
#: output directory to save log and model checkpoints, should be GCS path with TPU
output_path = None
# output_path = "/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/result"

#: training params
epochs = 50
learning_rate = 5e-5
min_learning_rate = 1e-5
warmup_rate = 0.06
warmup_steps = None
batch_size = 64  #128
dev_batch_size = 128  #512
num_valid_dataset = 500
tensorboard_update_freq = 1

#: device to use (TPU or GPU or CPU)
device = "TPU"
#: Use mixed precision FP16
mixed_precision = True  #False
#: Set random seed
seed = 42

In [6]:
my_config = {
    "pretrained_model": pretrained_model,
    'epochs': epochs,
    "learning_rate": learning_rate,
    "min_learning_rate": min_learning_rate,
    "warmup_rate": warmup_rate,
    "warmup_steps": warmup_steps,
    "batch_size": batch_size,
    "dev_batch_size": dev_batch_size,
    "num_valid_dataset": num_valid_dataset,
    "tensorboard_update_freq": tensorboard_update_freq
}

In [7]:
if output_path is not None and output_path.startswith("gs://"):
  from google.colab import auth
  auth.authenticate_user()

In [8]:
import pandas as pd
train_df = pd.read_csv(train_dataset_path)
train_df

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none
...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none


In [9]:
train_df.comment.tolist()

['김태리 정말 연기잘해 진짜',
 '공효진 발연기나이질생각이읍던데 왜계속주연일까',
 '누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에 감사하고 항상 좋은일만 가득하길~',
 '일본 축구 져라',
 '난 절대로 임현주 욕하는인간이랑은 안논다 @.@',
 '만진건 변하지 않는다 아이돌은 아이돌 좋아하는 여자들한테 이미지가 생명인데 얜 바닥쳤고 같이하는 멤버들도 끼리끼리로 보일듯ㅋㅋㅋㅋ',
 '연예계에 외국인노동자 많네..',
 '아이는 대부분 엄마가 원해서낳고 독박육아하고남편은 그냥 따라고는 수준!싫은 내색도 못하고 심지어 부담스러워하더라!',
 '먼기사를 기대하고 사진을 올리는지....관종',
 '개그맨 김경민 닮은다고 나만 느낌?',
 '역시 머리빨도 무시 못하는거임',
 '한동안 곧휴가 즐겁겠구나 부럽다 만석아',
 '이정도면 드라마가 아니라 블럭 버스터급영화~~ 왠쥐 베본2가 나올지도.',
 '국민들 대다수가 김미화씨의 진행능력에 감탄했다는데? 여기 비판하는 댓글은 다 블랙리스트 옹호충인가?',
 '홍자가 그말을 진심으로 했다는 식으로 몰아가네...?!전라도는 상종을 못할 인갈들.',
 '서울대 이혜성 >>>>>> 넘사 >>>>>>> 동덕여대 한혜진. 전현무도 결혼할 나인데 현명하게 판단한거임',
 '김지원빼고 다른 배우들 다 발연기같음....',
 '우리나라 의학기술의 발달은 실로 놀랍다',
 '결혼 축하합니다~ 행복하세요',
 '월트디즈니 만화를 보는듯...',
 '박명수는? 솔직히 존재의 이유를 못찾겠네',
 '솔직히 개듣보잡 아님?',
 '무슨상관이냐고 기자양반ㅠ',
 '경규옹이 저 맥락에서 비교 대상을 차은우로 한건 그만큼 차은우를 대표적인 미남이라고 생각한다는 방증이지',
 '살빼니까 왜케 느끼하지ㅋㅋ',
 '장도연 양세형 추가 투입 케미 좋아요 나래와 좋아요',
 '류준열 좀 그만....',
 '연기라고해도 그거 정말 남편 입장에서 굉장한 트라우마지..',
 '여자가 나이 많으면 이혼율 높은거 RG?',


In [10]:
# !pip install koco

In [11]:
# import koco

# train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')
# train_df = train_dev['train']

In [12]:
# len(train_dev['train'])

In [13]:
# len(train_dev['dev'])

In [14]:
# ttt = koco.load_dataset('korean-hate-speech', mode='test')
# len(ttt)

In [15]:
def load_dataset(data, tokenizer: AutoTokenizer, shuffle: bool = False, data_type='csv') -> tf.data.Dataset:
    """
    Load Hate Speech dataset from local file or web

    :param dataset_path: local file path or file uri
    :param tokenizer: PreTrainedTokenizer for tokenizing
    :param shuffle: whether shuffling lines or not
    :returns: Hate Speech dataset, number of dataset
    """
    # if dataset_path.startswith("https://"):
    #     with urllib.request.urlopen(dataset_path) as response:
    #         data = response.read().decode("utf-8")
    # else:
    #     with open(dataset_path) as f:
    #         data = f.read()
    # lines = data.splitlines()[1:]
    # if shuffle:
    #     random.shuffle(lines)

    bos = tokenizer.bos_token
    eos = tokenizer.eos_token

    bias_label2id = {"none": 0, "gender": 1, "others": 2}
    hate_label2id = {"none": 0, "hate": 1, "offensive": 1}

    title = []
    sentences = []
    bias_labels = []
    hate_labels = []
    if data_type == 'csv':
        bias_list = data.bias.tolist()
        hate_list = data.hate.tolist()
    # for comment, _, bias_label, hate_label in csv.reader(lines, delimiter="\t"):
    #     sentences.append(bos + comment + eos)
    #     bias_labels.append(bias_label2id[bias_label])
    #     hate_labels.append(hate_label2id[hate_label])
        title = data.title.tolist()
        sentences = data.comment.tolist()

        for i in range(len(sentences)):
            title[i] = bos + title[i] + eos
            sentences[i] = bos + sentences[i] + eos
            bias_labels.append(bias_label2id[bias_list[i]])
            hate_labels.append(hate_label2id[hate_list[i]])
    else:
        for i in range(len(data)):
            title.append(bos + data[i]['news_title'] + eos)
            sentences.append(bos + data[i]['comments'] + eos)
            bias_labels.append(bias_label2id[data[i]['bias']])
            hate_labels.append(hate_label2id[data[i]['hate']])

    inputs = dict(
        tokenizer(
            title, sentences,
            padding=True,
            return_tensors="tf",
            return_token_type_ids=False,
            return_attention_mask=True,
        )
    )

    dataset = tf.data.Dataset.from_tensor_slices((inputs, {"bias": bias_labels, "hate": hate_labels}))
    return dataset

In [16]:
if seed:
    set_random_seed(seed)

In [17]:
strategy = get_device_strategy(device)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.82.25.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.82.25.138:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Mixed Precision

In [18]:
with strategy.scope():
    if mixed_precision:
        mixed_type = "mixed_bfloat16" if device == "TPU" else "mixed_float16"
        policy = tf.keras.mixed_precision.experimental.Policy(mixed_type)
        tf.keras.mixed_precision.experimental.set_policy(policy)

# Load Dataset

In [19]:
# from sklearn.model_selection import train_test_split
                                                         
# train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)

In [20]:
with strategy.scope():
        tokenizer = AutoTokenizer.from_pretrained(pretrained_tokenizer, use_auth_token=use_auth_token)

        dataset = load_dataset(train_df, tokenizer, True)
        # dataset = load_dataset(train_df, tokenizer, True, data_type='url')
        train_dataset = dataset.skip(num_valid_dataset).batch(batch_size)
        valid_dataset = dataset.take(num_valid_dataset).batch(dev_batch_size)
        # dev_dataset = load_dataset(val_data, tokenizer).batch(dev_batch_size)

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
len(dataset)

8367

In [22]:
len(train_dataset)

123

In [23]:
len(valid_dataset)

4

# Load Model

In [24]:
with strategy.scope():
    model = TFBartForSequenceMultiClassification.from_pretrained(
    # model = BartForSequenceClassification.from_pretrained(
        pretrained_model,
        list_num_labels={"bias": 3, "hate": 2},
        use_auth_token=use_auth_token,
        from_pt=from_pytorch,
    )

Downloading:   0%|          | 0.00/155M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceMultiClassification: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing TFBartForSequenceMultiClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceMultiClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBartForSequenceMultiClassification were not initialized from the PyTorch model and are newly initialized: ['classification_head0.dense.weight', 'classification_head0.dense.bias', 'classification_head0.out_proj.weight', 'classification_head0.out_proj.bias', 'classification_head1.dense.weight', 'cl

In [25]:
print(model)

# Model Compile

In [26]:
with strategy.scope():
    model.compile(
        optimizer=AdamWeightDecay(
            LRScheduler(
                len(train_dataset) * epochs,
                learning_rate,
                min_learning_rate,
                warmup_rate,
                warmup_steps,
            ),
            weight_decay_rate=0.01,
            exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"],
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    )

# Model Training

In [27]:
# wandb.init(project='Multi-label Comment Classification(Tensor Flow)', 
#            entity='seongjin_ahn', 
#            name=my_config['pretrained_model'],
#            config=my_config
#            )
# wandb.watch(model, criterion, log='all', log_freq=10)  

In [28]:
with strategy.scope():
    model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint(
                path_join(output_path, "best_model.ckpt"),
                save_weights_only=True,
                save_best_only=True,
                monitor="val_loss",
                mode="min",
                verbose=1,
            ),
            tf.keras.callbacks.TensorBoard(
                path_join(output_path, "logs"), update_freq=tensorboard_update_freq
            ),
            # WandbCallback()
        ] if output_path is not None else None,
    )

Epoch 1/50
123/123 [==============================] - 83s 220ms/step - loss: 1.5552 - bias_loss: 0.8827 - hate_loss: 0.6725 - bias_accuracy: 0.6539 - hate_accuracy: 0.5995 - val_loss: 1.4345 - val_bias_loss: 0.8085 - val_hate_loss: 0.6259 - val_bias_accuracy: 0.6360 - val_hate_accuracy: 0.6400
Epoch 2/50
123/123 [==============================] - 9s 70ms/step - loss: 1.3000 - bias_loss: 0.7275 - hate_loss: 0.5726 - bias_accuracy: 0.7017 - hate_accuracy: 0.6972 - val_loss: 1.1684 - val_bias_loss: 0.6189 - val_hate_loss: 0.5489 - val_bias_accuracy: 0.7340 - val_hate_accuracy: 0.7140
Epoch 3/50
123/123 [==============================] - 9s 69ms/step - loss: 1.1471 - bias_loss: 0.6230 - hate_loss: 0.5242 - bias_accuracy: 0.7444 - hate_accuracy: 0.7397 - val_loss: 1.1048 - val_bias_loss: 0.5641 - val_hate_loss: 0.5406 - val_bias_accuracy: 0.7640 - val_hate_accuracy: 0.7320
Epoch 4/50
123/123 [==============================] - 11s 86ms/step - loss: 1.0248 - bias_loss: 0.5554 - hate_loss: 0.4

# Model Evaluate

In [29]:
# with strategy.scope():
#     total_loss, bias_loss, hate_loss, bias_accuracy, hate_accuracy = model.evaluate(dev_dataset)

In [30]:
# test_dataset_path = "/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/data/test.csv"
test_df = pd.read_csv(test_dataset_path)
test_df

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...
...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...


In [31]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/data/sample_submission.csv')
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,none
3,3,none,none
4,4,none,none
...,...,...,...
506,506,none,none
507,507,none,none
508,508,none,none
509,509,none,none


In [32]:
test_df['bias'] = submit['bias']
test_df['hate'] = submit['hate']
test_df
test_dataset = load_dataset(test_df, tokenizer).batch(dev_batch_size)

In [33]:
pred = model.predict(test_dataset)
pred

{'bias': array([[7.28125, -4.96875, -9.6875],
        [2.26562, -11.875, 3.84375],
        [10, -9.5625, -5.03125],
        ...,
        [-4.9375, 5.96875, -1.54688],
        [0.722656, -11.8125, 5.65625],
        [-0.78125, -2.57812, -3.04688]], dtype=bfloat16),
 'hate': array([[4.65625, -12.5],
        [-7.5625, 1.375],
        [-12.9375, 6.75],
        ...,
        [-11.4375, 10.8125],
        [-5.125, -1.27344],
        [-8.5, 5.3125]], dtype=bfloat16)}

In [34]:
import numpy as np

def softmax(a) :
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

print(softmax(pred['bias'][0]).argmax())

0


In [35]:
len(pred['bias'])

511

In [36]:
# 0-5 사이의 라벨 값 별로 bias, hate로 디코딩 하기 위한 딕셔너리
bias_dict = {0: 'none', 1: 'gender', 2: 'others'}
hate_dict = {0: 'none', 1: 'hate'}

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred_bias = ['' for i in range(len(submit))]
pred_hate = ['' for i in range(len(submit))]

for i in range(len(submit)):
    pred_bias[i]=str(bias_dict[softmax(pred['bias'][i]).argmax()])
    pred_hate[i]=str(hate_dict[softmax(pred['hate'][i]).argmax()])

print('decode Completed!')

decode Completed!


In [37]:
submit['bias'] = pred_bias
submit['hate'] = pred_hate
submit

,ID,bias,hate
0,0,none,none
1,1,others,hate
2,2,none,hate
3,3,none,hate
4,4,none,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,gender,hate
509,509,others,hate


In [38]:
submit.to_csv("/content/drive/MyDrive/Colab Notebooks/YEARDREAM/220223_4차 팀 모의 캐글/result/submission_21_bart_ko_small_batch64_128_valnum500.csv", index=False)